# Experiments on getting prices
* As quickly as possible!

# Findings
* When market is `open`:
    - use `executeAsync` from engine with `price` coro to get prices
    - this is fast
* When market is `closed`:
    - use `ba_async` to get a df of bid-ask-last price
    - this is slow. So data should be trimmed appropriately!


In [1]:
import pathlib
import datetime
import pandas as pd
import numpy as np
import asyncio

from ib_insync import IB, Stock, Contract, util

from collections import defaultdict
from tqdm import tqdm
from datetime import datetime, timedelta

from engine import Vars

In [2]:
util.startLoop()

In [15]:
# * INPUTS
MARKET = 'NSE'
SYMBOL = 'RELIANCE'
DTE = 10

In [16]:
# * SETTINGS
ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
locals().update(ibp.__dict__)

# set and empty log file
logf = pathlib.Path.cwd().joinpath('data', 'log', 'temp.log')
util.logToFile(path=logf, level=30)
with open(logf, "w"):
    pass

datapath = pathlib.Path.cwd().joinpath('data', MARKET.lower())

In [17]:
df_opts = pd.read_pickle(datapath.joinpath('df_opts.pkl'))
df_unds = pd.read_pickle(datapath.joinpath('df_unds.pkl'))
df_chains = pd.read_pickle(datapath.joinpath('df_chains.pkl'))

## Preparing contracts
### Nearest DTE contracts - sorted by strikeDelta

In [ ]:
df_sym = df_opts[df_opts.symbol == SYMBOL] # filter symbol
df_dte = df_sym[df_sym.dte == df_sym.dte.unique().min()] # filter dte

# sort from strike nearest to undPrice to farthest
df_dte = df_dte.iloc[abs(df_dte.strike-df_dte.undPrice.iloc[0]).argsort()]
contracts = df_dte.contract.unique()

contracts = contracts[:200] # !!! DATA LIMITER

df_dte.head()

### Option strike closest to underlying

In [ ]:
df1 = df_dte[:1]
contract = df1.contract.iloc[0]

df1

# Getting prices

## A) From Market data
### 1) Engine price
#### a) Single contract - with 8 second fill delay

#### b) Multiple contracts - with 8 second fill delay
* Takes about 10 seconds for 200 prices
* **`NOTE:`**
    - should be used only when the market is open
    - only gives `last` price, when the market is closed!
    - this `last` price is not accurate.

In [ ]:
%%time
from engine import price, pre_process, make_name, executeAsync, post_df

with IB().connect(HOST, PORT, CID) as ib:
    
    ib.client.setConnectOptions('+PACEAPI')
    
    df = ib.run(
            executeAsync(
                ib=ib,
                algo=price,
                cts = contracts,
                CONCURRENT = 200,
                TIMEOUT=8,
                post_process=post_df,
                SHOW_TQDM=True,
                **{'FILL_DELAY': 8}
            ))

In [ ]:
# sort by closest to strike
df = df.assign(undPrice=df_unds[df_unds.symbol == SYMBOL].undPrice.iloc[0])
df = df.iloc[abs(df.strike-df.undPrice.iloc[0]).argsort()]

# remove options without time
df1 = df[~df.time.isnull()]
df1

## B) From Historical Data

### 1) Bid-Ask for multiple contracts

**`NOTE`**
* This function is very slow, but gives bid-ask-last price when market is closed
* Takes 2 minutes for 20 prices!

In [ ]:
async def ba_async(ib, contracts):
   
    async def coro(c):
        
        lastPrice = await ib.reqHistoricalDataAsync(
                            contract=c,
                            endDateTime='',
                            durationStr='30 S',
                            barSizeSetting='30 secs',
                            whatToShow='TRADES',
                            useRTH=False,
                            formatDate=2,
                            keepUpToDate=False,
                            timeout=0)
        
        bidPrice = await ib.reqHistoricalDataAsync(
                            contract=c,
                            endDateTime='',
                            durationStr='30 S',
                            barSizeSetting='30 secs',
                            whatToShow='BID',
                            useRTH=False,
                            formatDate=2,
                            keepUpToDate=False,
                            timeout=0)
        
        askPrice = await ib.reqHistoricalDataAsync(
                            contract=c,
                            endDateTime='',
                            durationStr='30 S',
                            barSizeSetting='30 secs',
                            whatToShow='ASK',
                            useRTH=False,
                            formatDate=2,
                            keepUpToDate=False,
                            timeout=0)
        
        try:
            date = lastPrice[-1].date
            last = lastPrice[-1].close

        except IndexError:
            date = pd.NaT
            last = np.nan
            
        try:
            bid = bidPrice[-1].close
        except IndexError:
            bid = np.nan
            
        try:
            ask = askPrice[-1].close
        except IndexError:
            ask = np.nan
            
        return {c.conId: {'date': date, 'last': last, "bid": bid, "ask": ask}}
    
    tsks= [asyncio.create_task(coro(c), name=c.conId) for c in contracts]
    tasks = [await f for f in tqdm(asyncio.as_completed(tsks), total=len(tsks))]
    
    d = defaultdict(dict)
    for t in tasks:
        for k, v in t.items():
            d[k]=v
            
    df = pd.DataFrame(d).T
    
    return df

In [ ]:
%%time

ba_contracts = df1.contract[:3] # !!! DATA LIMITER

with IB().connect(HOST, PORT, CID) as ib:
    
    ib.client.setConnectOptions('+PACEAPI')
    
    df_ba = ib.run(ba_async(ib, ba_contracts))

## 2) Ask `trade` prices from history

In [ ]:
def onBarUpdate(bars, hasNewBar):
    print(bars[-1])

In [ ]:
with IB().connect(HOST, PORT, CID) as ib:
    bars = ib.reqRealTimeBars(contract, 5, 'TRADES', False)
    bars.updateEvent += onBarUpdate

In [ ]:
ib.cancelRealTimeBars(bars)

In [ ]:
async def hist_async(ib, contracts):
        
    # request data from start date
    async def coro(c):

        start = (datetime.utcnow()- timedelta(days=1)).date()
        end = datetime.utcnow().date()

        barsList = []
        dt = end

        while dt > start:

            bars = await ib.reqHistoricalDataAsync(contract,
                                            endDateTime=dt,
                                            durationStr='1 D',
                                            barSizeSetting='1 day',
                                            whatToShow='TRADES',
                                            useRTH=True,
                                            formatDate=2)
            if not bars:
                break
            barsList.append(bars)
            dt = bars[0].date

        return bars

    hist = await asyncio.gather(*[coro(c) for c in contracts])

    return hist

In [ ]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    
    ib.client.setConnectOptions('+PACEAPI')
    
    bars = ib.run(hist_async(ib, ba_contracts))